In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp crossing

In [ ]:
from chewc.core import *
from chewc.meiosis import *
import torch

## Crossing
> Using gamete operations to produce new individuals

In [ ]:
#| export
from chewc.core import *
from chewc.meiosis import *
import torch
def double_haploid_operation(gamete):
    return gamete.repeat(2, 1, 1)

def make_DH(genetic_map, individual):
    haploid_line = simulate_gametes(genetic_map, individual)
    return double_haploid_operation(haploid_line)

In [ ]:
#| export

def cross_operation(egg, pollen):
    return torch.stack((egg, pollen), dim=0).squeeze(1)

def make_cross(genetic_map, mother, father):
    egg = simulate_gametes(genetic_map, mother)  # Simulating gametes for the mother
    pollen = simulate_gametes(genetic_map, father)  # Simulating gametes for the father
    return cross_operation(egg, pollen)

In [ ]:
from chewc.core import *
from chewc.meiosis import *
import torch
ploidy = 2
number_chromosomes = 20
loci_per_chromosome = 10
n_founders = 50
genetic_map = create_random_genetic_map(number_chromosomes,loci_per_chromosome)
crop_genome = Genome(ploidy, number_chromosomes, loci_per_chromosome, genetic_map)
founder_pop = create_random_founder_pop(crop_genome , n_founders)



In [ ]:
#DH the first individual
print(make_DH(genetic_map, founder_pop[0]).shape)
#cross individual 2 and 3
print(make_cross(genetic_map, founder_pop[1], founder_pop[2]).shape)

torch.Size([2, 20, 10])
torch.Size([2, 20, 10])


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()